In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

data_folder = "data/utkface_aligned_cropped/UTKFace"
X_train = [] 
y_age_train = [] 


for filename in os.listdir(data_folder):
    if filename.endswith('.jpg'):
        # Extract the age, gender, race, and datetime from the filename
        
        age = filename[:-4].split('_')[0]
        # Load the image using OpenCV
        image_path = os.path.join(data_folder, filename)
        image = cv2.imread(image_path)
        
        # Preprocess the image if necessary (e.g., resize, normalize, etc.)
        # ...
        
        # Append the image and age label to the lists
        X_train.append(image)
        y_age_train.append(int(age))

# Convert the lists to numpy arrays
X_train = np.array(X_train)
y_age_train = np.array(y_age_train)

# Print the shape of the arrays
print('Xtrain shape:', X_train.shape)
print('ytrain shape:', y_age_train.shape)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(9):
    x = np.random.randint(0, len(X_train))
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[x])
    plt.title(y_age_train[x])
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
"""
create a new label named y_age_stage
Infant and Toddler: 0-3 years   class = 0
Preschooler and Kindergartener: 4-6 years  class = 1
child: 7-12 years class = 2
adolescent: 13-21 years class = 3
Adult in the prime : 22-37 years class = 4
Middle adult: 38-65 years class = 5
Elderly: 66-84 years class = 6
Oldest old: 85 years and older class = 7
"""
def map_age_stage(age):
    if age <= 3:
        return 0
    elif age <= 6:
        return 1
    elif age <= 12:
        return 2
    elif age <= 21:
        return 3
    elif age <= 37:
        return 4
    elif age <= 65:
        return 5
    elif age <= 84:
        return 6
    else:
        return 7

y_stage_train = np.array([map_age_stage(age) for age in y_age_train])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X_train, y_age_train, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
train_data_normalized = torch.Tensor(X_train/255)
train_data_normalized = train_data_normalized.permute(0,3,1,2)
train_labels = torch.Tensor(y_train).long()
val_data_normalized = torch.Tensor(X_val/255)
val_data_normalized = val_data_normalized.permute(0,3,1,2)
val_labels = torch.Tensor(y_val).long()
test_data_normalized = torch.Tensor(X_test/255)
test_data_normalized = test_data_normalized.permute(0,3,1,2)
test_labels = torch.Tensor(y_test).long()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import skorch
from skorch import NeuralNetRegressor
from torchvision.models import resnet18


class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.resnet = resnet18(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 1)

    def forward(self, x):
        return self.resnet(x)
    
model = NeuralNetRegressor(
    module=ResNet,
    optimizer=optim.SGD,
    criterion=nn.MSELoss,
    lr=1e-4,
    max_epochs=50,
    batch_size=32,
    callbacks=[skorch.callbacks.EarlyStopping(lower_is_better=True)],
    device='cuda' if torch.cuda.is_available() else 'cpu'
    )

In [ ]:

model.fit(np.asarray(train_data_normalized), torch.tensor(np.asarray(train_labels)).float())


In [ ]:
test_preict = model.predict(np.asarray(test_data_normalized))
test_preict = np.asarray(test_preict)
test_labels = np.asarray(test_labels)
MAE = np.mean(np.abs(test_preict - test_labels))
print(MAE)